<a href="https://colab.research.google.com/github/BrunoSDomingues/projeto-iris/blob/main/projeto_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de detecção de íris

## Limpando o ambiente

In [1]:
!rm -rf G6_iris_recognition/
!rm -rf enhanced_imgs/
!rm -rf images/
!rm -rf model.pickle

## Baixando bibliotecas e imagens a serem utilizadas

In [2]:
!pip3 install scikit-image --upgrade

In [3]:
!git clone https://github.com/lucianosilva-github/G6_iris_recognition.git

Cloning into 'G6_iris_recognition'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 31 (delta 3), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [4]:
!git clone https://github.com/lucianosilva-github/images.git
!rm -rf images/.git/

Cloning into 'images'...
remote: Enumerating objects: 1266, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1266 (delta 1), reused 0 (delta 0), pack-reused 1246
Receiving objects: 100% (1266/1266), 189.65 MiB | 23.62 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (1163/1163), done.


In [5]:
# Imports
import cv2
import os
import G6_iris_recognition as g6
import matplotlib.pyplot as plt
import numpy as np

## Tratando as imagens

### Melhorando a nitidez

In [ ]:
# Funcao que retorna uma imagem mais nitida usando uma mascara de nitidez
# Obtida em https://stackoverflow.com/a/55590133/9771897
def sharpen_image(img, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """
    img = imagem de entrada
    kernel_size = tamanho do kernel
    sigma = valor de sigma a ser usado no GaussianBlur
    amount = intensidade da nitidez: quanto maior, mais nitida a imagem
    threshold = limite a ser aplicado na mascara de baixo contraste
    """
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)
    sharpened = float(amount + 1) * img - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    
    if threshold > 0:
        low_contrast_mask = np.absolute(img - blurred) < threshold
        np.copyto(sharpened, img, where=low_contrast_mask)
        
    return sharpened

### Melhorando o brilho e o contraste

De acordo com a [documentação oficial](https://docs.opencv.org/master/d3/dc1/tutorial_basic_linear_transform.html) do OpenCV, a melhor forma de fazer ajustes no brilho e no contraste é utilizando a fórmula abaixo

$g(x) = \alpha \cdot f(x) + \beta$

onde: 
- $\alpha$ é o parâmetro de _gain_ (utilizado para controlar o contraste);
- $\beta$ é o parâmetro de _bias_ (utilizado para controlar o brilho);

Como estamos tratando de uma imagem, a conta de ajuste deve ser feita para cada pixel. Assim, podemos adaptar a fórmula:

$img_{out} (i, j) = \alpha \cdot img_{in} (i, j) + \beta$

Para ajustes no contraste, basta alterar o valor de $\alpha$:
- para $\alpha = 1$, a imagem não tem alteração de contraste;
- para $0 < \alpha < 1$, o contraste da imagem é reduzido;
- para $\alpha > 1$, o contraste da imagem é intensificado;

Para ajustes no brilho, basta alterar o valor de $\beta$:
- para $\beta = 0$, a imagem não tem alteração de brilho;
- para $\beta < 0$, o brilho da imagem é reduzido;
- para $\beta > 0$, o brilho da imagem é intensificado;

De modo a fazer esta conta, devemos iterar por cada pixel e armazenar o valor calculado em uma nova matriz de imagem. Para tal, podemos usar a função [`convertScaleAbs`](hhttps://docs.opencv.org/4.5.3/d2/de8/group__core__array.html#ga3460e9c9f37b563ab9dd550c4d8c4e7d) do `cv2`, que faz a conta automaticamente.

In [ ]:
# Funcao que ajusta simultaneamente o brilho e o contraste
def adjust_brightness_and_contrast(img, alpha=1, beta=0, gamma=1):
    """
    img = imagem de entrada
    alpha = valor de contraste
    beta = valor do brilho
    """
    
    alpha = float(alpha)
    beta = int(beta)
    gamma = float(gamma)
    
    # Se o ajuste de contraste for 1 e o ajuste de brilho for 0, a imagem sera inalterada
    if alpha == 1 and beta == 0:
        return img
    
    else:
        if alpha < 0:
            raise ValueError("Alpha value can't be lower than 0")
        
        else:
            new_img = np.zeros(img.shape, img.dtype)
            new_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
                        
    # Gamma correction
    if gamma == 1:
        return new_img
    
    else:
        lookUpTable = np.empty((1,256), np.uint8)
        
        for i in range(256):
            lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
            
        lookup = cv2.LUT(new_img, lookUpTable)
        corrected = cv2.hconcat([new_img, lookup])
        
        return corrected

### Tratando e armazenando as novas imagens

Foi feito um teste utilizando a técnica de equalização do histograma de cores da imagem; no entanto, este gerou imagens com muito brilho nas pupilas e pouco destaque para as íris. Assim, realizou-se apenas ajustes na nitidez, no contraste e no brilho das imagens.

Com equalização            |  Sem equalização
:-------------------------:|:-------------------------:
![](https://github.com/BrunoSDomingues/projeto-iris/blob/main/with_eq.bmp?raw=1)           |![](https://github.com/BrunoSDomingues/projeto-iris/blob/main/without_eq.bmp?raw=1)

In [ ]:
# Gerando pasta para salvar as imagens tratadas
if not os.path.isdir("enhanced_imgs"):
    os.mkdir("enhanced_imgs")

In [ ]:
# Gerando lista com os numeros das pastas
img_list = [f"{str(i).zfill(4)}" for i in range(60) if i != 12] # Nao existe pasta 0012

# A maior parte das pastas contém 20 imagens cada
n_imgs = 20

In [ ]:
# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    # Checa se a pasta nao existe, e cria ela se nao existir
    if not os.path.isdir(enhanced_path):
        os.mkdir(os.path.join("enhanced_imgs", path))
        
    # Percorrendo cada imagem dentro da pasta
    for i in range(n_imgs):
        # Obtem o path completo da imagem
        filename = f"{path}_{str(i).zfill(3)}.bmp"
        img_path = os.path.join("images", path, filename)        
        
        # Existem casos em que uma pasta nao possui 20 imagens
        # Verifica-se portanto se a imagem existe antes de processar
        if not os.path.isfile(img_path):
            pass
        
        else:
            img = cv2.imread(img_path)
            
            # Tratando
            sharpened = sharpen_image(img, amount=2.0)
            processed = adjust_brightness_and_contrast(sharpened, alpha = 2.2, beta = 30, gamma = 0.45)

            # Salvando a imagem
            path_processed = os.path.join("enhanced_imgs", path, filename)
            cv2.imwrite(path_processed, processed)

## Treinando o modelo

In [ ]:
open("model.pickle", "a").close()
g6.iris_model_train("images", "model.pickle")

directory_list ['images/.git/logs/refs/remotes/origin', 'images/.git/logs/refs/remotes', 'images/.git/logs/refs/heads', 'images/.git/logs/refs', 'images/.git/refs/remotes/origin', 'images/.git/refs/remotes', 'images/.git/refs/heads', 'images/.git/refs/tags', 'images/.git/objects/info', 'images/.git/objects/pack', 'images/.git/info', 'images/.git/logs', 'images/.git/hooks', 'images/.git/refs', 'images/.git/objects', 'images/.git/branches', 'images/0014', 'images/0047', 'images/0034', 'images/0030', 'images/0016', 'images/0052', 'images/0054', 'images/0053', 'images/0035', 'images/0059', 'images/0022', 'images/0056', 'images/0008', 'images/0023', 'images/0011', 'images/0021', 'images/0042', 'images/0006', 'images/0009', 'images/0046', 'images/0038', 'images/0010', 'images/0025', 'images/0041', 'images/0051', 'images/0029', 'images/0013', 'images/0000', 'images/0027', 'images/0037', 'images/0018', 'images/0031', 'images/0003', 'images/0005', 'images/0040', 'images/0048', 'images/0032', 'i

KeyboardInterrupt: ignored

## Testando o modelo

In [ ]:
# Contadores
total_imgs = 0
matches = 0
failed = 0

# Matriz de confusao
confusion_matrix = []

# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    for image in os.listdir(enhanced_path):
        # Contando o numero de imagens total
        total_imgs += 1
        
        # Pegando o path completo do arquivo
        img_path = os.path.join(enhanced_path, image)
        
        # Passando a imagem para o modulo G6
        guess = g6.iris_model_test("model.pickle", img_path)
#         temp = guess
        
#         # Se nao deu match, o valor na matriz de confusao deve ser -1
#         if guess == "unmatch":
#             failed += 1
#             temp = -1
        
#         # Se der match, o guess tem que bater com o numero da pasta
#         elif int(guess) == int(path):
#             matches += 1
        print(guess)
        
#         confusion_matrix.append(np.array([int(path), temp]))
        
# confusion_matrix = np.array(confusion_matrix)

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment


locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
unmatch
locate expression 1 local variable 'east_mark' referenced before assignment


KeyboardInterrupt: 

In [ ]:
print(f"Matches: {matches}")
print(f"Failures: {failed}")
print(f"Success rate: {round(matches/total_imgs*100, 2)}%")

Matches: 0
Failures: 1163
Success rate: 0.0%


## Análise dos resultados

Embora as imagens tenham sido tratadas para deixar a íris mais destacada, o modelo não foi capaz de detectar nenhuma imagem com sucesso. Uma possível explicação para tal é que o módulo `G6_iris_recognition` tem como requerimento que no mínimo 90% da íris esteja visível, e a maior parte das imagens contém a íris parcialmente oculta, seja pela pupila ou por fios de cabelo que estão na frente da íris. Outro fator que pode explicar este resultado é que os tons de cinza presentes nas imagens não são tão distintos entre si, o que pode confundir o treinamento.

![](https://github.com/BrunoSDomingues/projeto-iris/blob/main/requirements.png?raw=1)